In [7]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.utils import to_categorical

# Function to load and preprocess images from a specific subfile
def load_and_preprocess_images_from_subfile(directory, subfile):
    images = []
    labels = []

    label = int(subfile) if subfile.isdigit() else None  # Extract label from the subfile
    
    if label is not None:
        subfile_dir = os.path.join(directory, subfile)

        for filename in os.listdir(subfile_dir):
            if filename.endswith(".png"):
                img_path = os.path.join(subfile_dir, filename)
                
                img = cv2.imread(img_path)
                img = cv2.resize(img, (150, 150))  # Resize images to 150x150
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
                img = img / 255.0  # Normalize pixel values to [0, 1]
                
                images.append(img)
                labels.append(label)

    print(f"Number of images in {directory}/{subfile}: {len(images)}")
    if len(images) == 0:
        print(f"No images found in {directory}/{subfile}")

    return np.array(images), np.array(labels)

# Function to load and preprocess all subfiles for a hallmark
def load_all_subfiles(hallmark_dir):
    X_all, y_all = [], []
    
    for subfile in os.listdir(hallmark_dir):
        if subfile.isdigit() or subfile.lower() == 'ignore':
            X_subfile, y_subfile = load_and_preprocess_images_from_subfile(hallmark_dir, subfile)
            X_all.extend(X_subfile)
            y_all.extend(y_subfile)

    return np.array(X_all), np.array(y_all)

steatosis_dir = 'D:/DATASET/CNN/steatosis/train'
inflammation_dir = 'D:/DATASET/CNN/inflammation/train'
ballooning_dir = 'D:/DATASET/CNN/ballooning/train'
fibrosis_dir = 'D:/DATASET/CNN/fibrosis/train'

# Load and preprocess images for each hallmark
X_steatosis, y_steatosis = load_all_subfiles(steatosis_dir)
X_inflammation, y_inflammation = load_all_subfiles(inflammation_dir)
X_ballooning, y_ballooning = load_all_subfiles(ballooning_dir)
X_fibrosis, y_fibrosis = load_all_subfiles(fibrosis_dir)

# One-hot encode labels
if len(y_steatosis) > 0:
    le = LabelEncoder()
    y_steatosis = le.fit_transform(y_steatosis)
    y_steatosis = to_categorical(y_steatosis)
    print(y_steatosis)

# One-hot encode labels
if len(y_inflammation) > 0:
    le = LabelEncoder()
    y_inflammation = le.fit_transform(y_inflammation)
    y_inflammation = to_categorical(y_inflammation)
    
# One-hot encode labels
if len(y_ballooning) > 0:
    le = LabelEncoder()
    y_ballooning = le.fit_transform(y_ballooning)
    y_ballooning = to_categorical(y_ballooning)

# One-hot encode labels
if len(y_fibrosis) > 0:
    le = LabelEncoder()
    y_fibrosis = le.fit_transform(y_fibrosis)
    y_fibrosis = to_categorical(y_fibrosis)

Number of images in D:/DATASET/CNN/steatosis/train/0: 3812
Number of images in D:/DATASET/CNN/steatosis/train/1: 22752


MemoryError: Unable to allocate 3.81 GiB for an array with shape (22752, 150, 150) and data type float64

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.utils import to_categorical

input_shape = (150, 150, 3)

# Define the number of classes based on your dataset
num_classes_steatosis = 4 
num_classes_inflammation = 3  
num_classes_ballooning = 2  
num_classes_fibrosis = 5  

# Create and compile the ResNet50 model for steatosis
base_model_steatosis = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
x_steatosis = base_model_steatosis.output
x_steatosis = GlobalAveragePooling2D()(x_steatosis)
x_steatosis = Dense(512, activation='relu')(x_steatosis)
predictions_steatosis = Dense(num_classes_steatosis, activation='softmax')(x_steatosis)

model_steatosis = Model(inputs=base_model_steatosis.input, outputs=predictions_steatosis)
model_steatosis.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_steatosis.save("D:/DATASET/Models/model_steatosis.h5")

# Create and compile model for inflammation
base_model_inflammation = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
x_inflammation = base_model_inflammation.output
x_inflammation = GlobalAveragePooling2D()(x_inflammation)
x_inflammation = Dense(512, activation='relu')(x_inflammation)
predictions_inflammation = Dense(num_classes_inflammation, activation='softmax')(x_inflammation)

model_inflammation = Model(inputs=base_model_inflammation.input, outputs=predictions_inflammation)
model_inflammation.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_inflammation.save("D:/DATASET/Models/model_inflammation.h5")

# Create and compile models for ballooning 
base_model_ballooning = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
x_ballooning = base_model_ballooning.output
x_ballooning = GlobalAveragePooling2D()(x_ballooning)
x_ballooning = Dense(512, activation='relu')(x_ballooning)
predictions_ballooning = Dense(num_classes_ballooning, activation='softmax')(x_ballooning)

model_ballooning = Model(inputs=base_model_ballooning.input, outputs=predictions_ballooning)
model_ballooning.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_ballooning.save("D:/DATASET/Models/model_ballooning.h5")

# Create and compile models for fibrosis
base_model_fibrosis = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
x_fibrosis = base_model_fibrosis.output
x_fibrosis = GlobalAveragePooling2D()(x_fibrosis)
x_fibrosis = Dense(512, activation='relu')(x_fibrosis)
predictions_fibrosis = Dense(num_classes_fibrosis, activation='softmax')(x_fibrosis)

model_fibrosis = Model(inputs=base_model_fibrosis.input, outputs=predictions_fibrosis)
model_fibrosis.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_fibrosis.save("D:/DATASET/Models/model_fibrosis.h5")

# Specify the new input shape based on the resized images, e.g., (150, 150, 1)
input_shape = (150, 150, 3)

# Create and compile the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
#predictions = Dense(num_classes_steatosis, activation='softmax')(x)
#predictions = Dense(num_classes_inflammation, activation='softmax')(x)
#predictions = Dense(num_classes_steatosis, activation='softmax')(x)
predictions = Dense(num_classes_fibrosis, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.save("D:/DATASET/Models/model_4.h5")

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 156, 156, 3)  0           ['input_5[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 75, 75, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 75, 75, 64)   256         ['conv1_conv[0][0]']       

 conv2_block3_2_conv (Conv2D)   (None, 38, 38, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 38, 38, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 38, 38, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 38, 38, 256)  16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_blo

 conv3_block3_2_conv (Conv2D)   (None, 19, 19, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 19, 19, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 19, 19, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 19, 19, 512)  66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_blo

 conv4_block2_1_relu (Activatio  (None, 10, 10, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 10, 10, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 10, 10, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 10, 10, 256)  0          ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
          

                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 10, 10, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 10, 10, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_block5_2_bn (BatchNormal  (None, 10, 10, 256)  1024       ['conv4_block5_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_2_relu (Activatio  (None, 10, 10, 256)  0          ['conv4_block5_2_bn[0][0]']      
 n)       

 conv5_block2_1_conv (Conv2D)   (None, 5, 5, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 5, 5, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 5, 5, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block2_2_conv (Conv2D)   (None, 5, 5, 512)    2359808     ['conv5_block2_1_relu[0][0]']    
                                                                                                  
 conv5_blo

In [3]:
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from keras.models import Model

# Define input shape for grayscale images
input_shape = (150, 150, 1)  # Adjust dimensions based on your resized image dimensions

# Input layer
input_layer = Input(shape=input_shape)

# Model architecture
x = Conv2D(64, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Add more Conv2D and MaxPooling2D layers as needed for your architecture
x = Conv2D(64, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Global Average Pooling layer
x = GlobalAveragePooling2D()(x)

# Fully connected layers
x = Dense(512, activation='relu')(x)

# Output layer
num_classes_steatosis = 4 
num_classes_inflammation = 3  
num_classes_ballooning = 2  

predictions = Dense(num_classes_steatosis, activation='softmax')(x)
predictions = Dense(num_classes_inflammation, activation='softmax')(x)
predictions = Dense(num_classes_ballooning, activation='softmax')(x)

# Create the model
model = Model(inputs=input_layer, outputs=predictions)

# Compile the model (replace optimizer and loss with your choices)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary to check the architecture
model.summary()

model.save("D:/DATASET/Models/model_5.h5")

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 150, 150, 1)]     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 148, 148, 64)      640       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 64)       0         
 2D)                                                             
                                                                 
 global_average_pooling2d_5   (None, 64)               0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_10 (Dense)            (None, 512)               33280     
                                                                 
 dense_13 (Dense)            (None, 2)                 1026

In [ ]:
# Load and preprocess images for each hallmark
X_steatosis, y_steatosis = load_all_subfiles(steatosis_dir)
X_inflammation, y_inflammation = load_all_subfiles(inflammation_dir)
X_ballooning, y_ballooning = load_all_subfiles(ballooning_dir)
X_fibrosis, y_fibrosis = load_all_subfiles(fibrosis_dir)

# Encode labels for each hallmark
label_encoders = {
    'steatosis': LabelEncoder(),
    'inflammation': LabelEncoder(),
    'ballooning': LabelEncoder(),
    'fibrosis': LabelEncoder()
}

# Fit and transform labels
y_steatosis = label_encoders['steatosis'].fit_transform(y_steatosis)
y_steatosis = to_categorical(y_steatosis, num_classes=num_classes_steatosis)

y_inflammation = label_encoders['inflammation'].fit_transform(y_inflammation)
y_inflammation = to_categorical(y_inflammation, num_classes=num_classes_inflammation)

y_ballooning = label_encoders['ballooning'].fit_transform(y_ballooning)
y_ballooning = to_categorical(y_ballooning, num_classes=num_classes_ballooning)

y_fibrosis = label_encoders['fibrosis'].fit_transform(y_fibrosis)
y_fibrosis = to_categorical(y_fibrosis, num_classes=num_classes_fibrosis)

# Split data into train and validation sets for each hallmark
X_steatosis_train, X_steatosis_val, y_steatosis_train, y_steatosis_val = train_test_split(
    X_steatosis, y_steatosis, test_size=0.2, random_state=42
)

X_inflammation_train, X_inflammation_val, y_inflammation_train, y_inflammation_val = train_test_split(
    X_inflammation, y_inflammation, test_size=0.2, random_state=42
)

X_ballooning_train, X_ballooning_val, y_ballooning_train, y_ballooning_val = train_test_split(
    X_ballooning, y_ballooning, test_size=0.2, random_state=42
)

X_fibrosis_train, X_fibrosis_val, y_fibrosis_train, y_fibrosis_val = train_test_split(
    X_fibrosis, y_fibrosis, test_size=0.2, random_state=42
)

# Train the model for each hallmark
model_steatosis.fit(X_steatosis_train, y_steatosis_train, validation_data=(X_steatosis_val, y_steatosis_val), epochs=5)
model_inflammation.fit(X_inflammation_train, y_inflammation_train, validation_data=(X_inflammation_val, y_inflammation_val), epochs=5)
model_ballooning.fit(X_ballooning_train, y_ballooning_train, validation_data=(X_ballooning_val, y_ballooning_val), epochs=5)
model_fibrosis.fit(X_fibrosis_train, y_fibrosis_train, validation_data=(X_fibrosis_val, y_fibrosis_val), epochs=5)

In [4]:
# Split data into train and validation sets

print("hello")

X_steatosis_train, X_steatosis_val, y_steatosis_train, y_steatosis_val = train_test_split(
    X_steatosis, y_steatosis, test_size=0.2, random_state=42
)

print("hello")

X_inflammation_train, X_inflammation_val, y_inflammation_train, y_inflammation_val = train_test_split(
    X_inflammation, y_inflammation, test_size=0.2, random_state=42
)

print("hello")

X_ballooning_train, X_ballooning_val, y_ballooning_train, y_ballooning_val = train_test_split(
    X_ballooning, y_ballooning, test_size=0.2, random_state=42
)

print("hello")

X_fibrosis_train, X_fibrosis_val, y_fibrosis_train, y_fibrosis_val = train_test_split(
    X_fibrosis, y_fibrosis, test_size=0.2, random_state=42
)

print("hello")

hello
hello
hello
hello
hello


In [5]:
# Train the model for each hallmark
model.fit(X_steatosis_train, y_steatosis_train, validation_data=(X_steatosis_val, y_steatosis_val), epochs=5)

model.fit(X_inflammation_train, y_inflammation_train, validation_data=(X_inflammation_val, y_inflammation_val), epochs=5)

model.fit(X_ballooning_train, y_ballooning_train, validation_data=(X_ballooning_val, y_ballooning_val), epochs=5)

model.fit(X_fibrosis_train, y_fibrosis_train, validation_data=(X_fibrosis_val, y_fibrosis_val), epochs=5)

Epoch 1/5


ValueError: in user code:

    File "C:\Users\JUHI\anaconda3\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\JUHI\anaconda3\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\JUHI\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\JUHI\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\JUHI\anaconda3\lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "C:\Users\JUHI\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Use

In [ ]:
from tensorflow.keras.models import Model

# Function to get the output of the last convolutional layer
def get_last_conv_layer_output(model):
    last_conv_layer_name = None
    for layer in reversed(model.layers):
        if 'conv' in layer.name:
            last_conv_layer_name = layer.name
            break

    if last_conv_layer_name:
        last_conv_layer_output = model.get_layer(last_conv_layer_name).output
        return last_conv_layer_output, last_conv_layer_name
    else:
        print("No convolutional layer found in the model.")
        return None, None

# Load your fibrosis model
model_fibrosis = "D:/DATASET/Models/model_4.h5"   # Load your fibrosis model here

# Get the output of the last convolutional layer
last_conv_layer_output, last_conv_layer_name = get_last_conv_layer_output(model_fibrosis)

if last_conv_layer_output is not None:
    print("Output of the last convolutional layer:", last_conv_layer_output)
    print("Name of the last convolutional layer:", last_conv_layer_name)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Model

# Function to get the output of the last convolutional layer
def get_activation_maps(model, img_path, layer_name):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (150, 150))  # Resize image to model input size
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB (assuming BGR input)
    img_array = np.expand_dims(img, axis=0) / 255.0  # Normalize pixel values
    
    intermediate_layer_model = Model(inputs=model.input,
                                     outputs=model.get_layer(layer_name).output)
    
    feature_maps = intermediate_layer_model.predict(img_array)
    return feature_maps, img

# Function to generate Class Activation Map (CAM)
def generate_cam(model, feature_maps, class_index, img):
    final_dense_weights = model.layers[-1].get_weights()[0]
    cam = np.dot(feature_maps, final_dense_weights[:, class_index])
    cam = cv2.resize(cam, (img.shape[1], img.shape[0]))
    cam = cam - np.min(cam)
    cam = cam / np.max(cam)
    return cam

# Load the trained fibrosis model
model_fibrosis = "D:/DATASET/Models/model_4.h5"  # Load your fibrosis model here

# Path to an example image for visualization
img_path = "D:/DATASET/CNN/fibrosis/train/3/9_44_12.png"
# Get the output of the last convolutional layer
layer_name = last_conv_layer_name
feature_maps, img = get_activation_maps(model_fibrosis, img_path, layer_name)

# Generate CAM for fibrosis class (assuming fibrosis class index is 0)
class_index_fibrosis = 0
cam_fibrosis = generate_cam(model_fibrosis, feature_maps, class_index_fibrosis, img)

# Plot the original image, feature map, and CAM
plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.imshow(img)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(feature_maps[0, :, :, 0], cmap='viridis')  # Display the first channel of the feature map
plt.title('Feature Map')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(img)
plt.imshow(cam_fibrosis, cmap='jet', alpha=0.5)  # Overlay CAM on the original image
plt.title('Class Activation Map (CAM) for Fibrosis')
plt.axis('off')

plt.tight_layout()
plt.show()